<a href="https://colab.research.google.com/github/poketmonster/sentimentanalysis/blob/master/ALC_SentimentAnalisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv, operator
import numpy as np
import nltk
import string
import re
import math
import random
import xml.etree.ElementTree as et
import pandas as pd
import time

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from random import shuffle
from nltk.stem.porter import *

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
!rm -r *
from google.colab import files

uploaded = files.upload() # then browse, select the files. It's then uploaded

Saving TASS2017_T1_development.xml to TASS2017_T1_development.xml
Saving TASS2017_T1_test.xml to TASS2017_T1_test.xml
Saving TASS2017_T1_training.xml to TASS2017_T1_training.xml


In [3]:
!rm -r *
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_development.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_test.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_training.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/ElhPolar_esV1.lex

--2019-04-14 08:17:30--  https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_development.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163475 (160K) [text/plain]
Saving to: ‘TASS2017_T1_development.xml’

TASS2017_T1_develop 100%[===================>] 159.64K  --.-KB/s    in 0.03s   

2019-04-14 08:17:30 (5.15 MB/s) - ‘TASS2017_T1_development.xml’ saved [163475/163475]

--2019-04-14 08:17:32--  https://raw.githubusercontent.com/poketmonster/sentimentanalysis/master/TASS2017_T1_test.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiti

In [6]:
#Cargar diccionario de polaridad

v = DictVectorizer(sparse=False)
s = open('ElhPolar_esV1.lex', 'r').read()


palabra = []
polaridad = []
with open('ElhPolar_esV1.lex') as f: 
  for line in f.readlines():
    div = line.split('\t')
    if len(div)==2:
      palabra.append(div[0])
      polaridad.append(div[1].replace("\n", ""))
  
Poldict = {}
for i in range(len(palabra)):
    Poldict[palabra[i]] = polaridad[i]
    
print(len(Poldict))
Poldict['abdicar']

4673


'negative'

In [0]:
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas DataFrame 
    with the given columns. The first element of df_cols is supposed to be 
    the identifier variable, which is an attribute of each node element in 
    the XML data; other features will be parsed from the text content of 
    each sub-element. """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    out_df = pd.DataFrame(columns = df_cols)
    
    for node in xroot: 
        res = []
        for el in df_cols[0:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index = True)
        
    return out_df

In [9]:
train = parse_XML("TASS2017_T1_training.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("TRAIN")
train.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
#print(train.head())
print("Langs: "+str(train.lang.unique()))
print(train['polarity'].value_counts(dropna=False))

dev = parse_XML("TASS2017_T1_development.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("DEV")
dev.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
#print(dev.head())
print("Langs: "+str(dev.lang.unique()))
print(dev['polarity'].value_counts(dropna=False))

test = parse_XML("TASS2017_T1_test.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("TEST")
test.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
#print(test.head())
print("Langs: "+str(test.lang.unique()))


TRAIN
Langs: ['es']
N       418
P       318
NONE    139
NEU     133
Name: polarity, dtype: int64
DEV
Langs: ['es']
N       219
P       156
NEU      69
NONE     62
Name: polarity, dtype: int64
TEST
Langs: ['es']


In [10]:
tknzr = TweetTokenizer()

trainCorpus = []
train_labels = []

devCorpus = []
dev_labels = []

for i in range(0,len(train)):
  senseurl = re.sub(r"http\S+", "", train.at[i, 'content'])
  tokenitzat = tknzr.tokenize(senseurl)
  trainCorpus.append(train.at[i, 'content'])
  train_labels.append(train.at[i,'polarity'])

for i in range(0,len(dev)):
  senseurl = re.sub(r"http\S+", "", dev.at[i, 'content'])
  tokenitzat = tknzr.tokenize(senseurl)
  devCorpus.append(dev.at[i, 'content'])
  dev_labels.append(dev.at[i,'polarity'])
  
print(len(trainCorpus))
print(len(train_labels))
print(len(devCorpus))
print(len(dev_labels))

#devCorpus

1008
1008
506
506


In [0]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation

spanish_stopwords = stopwords.words('spanish')

stemmer = SnowballStemmer('spanish')

non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))

stemmer = SnowballStemmer('spanish')
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed


def onwtokenizer(text):
  text = re.sub(r"http\S+", "", text)
  text = ''.join([c for c in text if c not in non_words])
  tokens =  word_tokenize(text)

  # stem
  try:
      stems = stem_tokens(tokens, stemmer)
  except Exception as e:
      print(e)
      print(text)
      stems = ['']
  return stems
  

In [0]:
# TfidfVectorizer CHAR
vectorizer = TfidfVectorizer(min_df=1,
                             max_df = 0.9,
                             sublinear_tf=True,
                             analyzer='char_wb',
                             ngram_range = (1,20),
                             max_features = None,
                             strip_accents = "unicode",
                             use_idf=True)


train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors[1])

  (0, 90759)	0.021646297666471075
  (0, 92073)	0.024171193599014328
  (0, 61809)	0.015282168306276525
  (0, 92074)	0.01833129221077423
  (0, 53208)	0.015528592946580664
  (0, 68581)	0.017481383547965138
  (0, 135451)	0.011127238062587542
  (0, 19244)	0.019407578324254528
  (0, 51194)	0.02010850370845873
  (0, 11066)	0.01951106173362051
  (0, 65583)	0.010649412351979066
  (0, 110962)	0.020847711534352453
  (0, 23038)	0.02285419671886908
  (0, 111022)	0.02119753926552142
  (0, 129690)	0.01829536300057581
  (0, 23050)	0.023288304611244502
  (0, 111023)	0.022279155140902653
  (0, 129691)	0.013673438930637084
  (0, 23051)	0.02388005162807004
  (0, 111024)	0.01381985119016975
  (0, 23052)	0.014649503252231648
  (0, 83590)	0.03944507598607009
  (0, 72747)	0.011341098106817767
  (0, 18283)	0.025222173475965823
  (0, 47883)	0.01289089510493581
  :	:
  (0, 18284)	0.03268624479032971
  (0, 83695)	0.0727087826919365
  (0, 44118)	0.06753564578811726
  (0, 18295)	0.03510313535604631
  (0, 83699)	0.0

In [57]:
# TfidfVectorizer WORDS
vectorizer = TfidfVectorizer(min_df=1,
                             max_df = 0.99,
                             tokenizer = onwtokenizer,
                             sublinear_tf=False,
                             analyzer='word',
                             ngram_range = (1,1),
                             max_features = None,
                             strip_accents = "ascii",
                             lowercase = True,
                             smooth_idf = True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)

print(vectorizer.get_feature_names())
print(train_vectors.shape)

print(train_vectors[1])

['a', 'aaahm', 'aaronnp', 'abac', 'abandon', 'abat', 'abces', 'aberron', 'abiert', 'abraz', 'abrazar', 'abrazart', 'abrazotee', 'abre', 'abres', 'abrig', 'abrilnapilotti', 'abrir', 'absolut', 'abstract', 'absurd', 'abuel', 'aburr', 'ac', 'acab', 'acapulcoshor', 'acc', 'acced', 'accion', 'ace', 'acept', 'acerc', 'acert', 'acevasociacion', 'achocoblog', 'achuch', 'aciert', 'acompan', 'acondicion', 'acongoj', 'acord', 'acos', 'acostumbr', 'activ', 'actor', 'actriz', 'actuacion', 'actualiz', 'acuerd', 'acust', 'adapt', 'adaptacion', 'adc', 'adem', 'adiccion', 'adioshi', 'adirect', 'adjet', 'adolescent', 'adopt', 'ador', 'adriansol', 'adribuz', 'adrimarinab', 'adsu', 'advert', 'advirti', 'afeit', 'afon', 'afor', 'afortun', 'agarotadipanem', 'agarzon', 'agenci', 'agmelendez', 'agnesfl', 'agobi', 'agost', 'agot', 'agradec', 'agreg', 'agresion', 'agresor', 'agu', 'aguant', 'ah', 'ahi', 'ahilodej', 'ahor', 'ahorapod', 'ahorr', 'ahurtadobuen', 'ai', 'aikoleru', 'ainhoamall', 'air', 'airatarr', '

In [0]:
# HashingVectorizer WORD
vectorizer = HashingVectorizer(n_features= 2 ** 22,
                             analyzer='word',
                             ngram_range = (1,1),
                             strip_accents = "ascii",
                             lowercase = True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape)

print(train_vectors[1])

(1008, 4194304)
  (0, 57778)	0.25
  (0, 203536)	0.25
  (0, 502360)	-0.25
  (0, 642616)	-0.25
  (0, 843554)	-0.25
  (0, 1157025)	0.25
  (0, 1300924)	0.25
  (0, 1347213)	0.25
  (0, 1585224)	0.25
  (0, 1665705)	0.25
  (0, 1863624)	-0.25
  (0, 2007332)	-0.25
  (0, 2312478)	-0.25
  (0, 2742171)	0.25
  (0, 3158444)	0.25
  (0, 3570582)	0.25


In [0]:
# HashingVectorizer CHAR
vectorizer = HashingVectorizer(n_features= 2 ** 18,
                             analyzer='char_wb',
                             ngram_range = (1,5),
                             strip_accents = "ascii",
                             lowercase = True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape)

print(train_vectors[1])

(1008, 262144)
  (0, 344)	0.020219563399637347
  (0, 1465)	0.020219563399637347
  (0, 1474)	-0.020219563399637347
  (0, 4203)	0.020219563399637347
  (0, 4211)	-0.020219563399637347
  (0, 4697)	0.020219563399637347
  (0, 5298)	0.040439126799274694
  (0, 5468)	0.020219563399637347
  (0, 6324)	0.020219563399637347
  (0, 7793)	-0.020219563399637347
  (0, 7841)	0.040439126799274694
  (0, 8294)	-0.040439126799274694
  (0, 8371)	-0.040439126799274694
  (0, 9661)	-0.020219563399637347
  (0, 10604)	-0.020219563399637347
  (0, 11389)	0.08087825359854939
  (0, 12136)	-0.020219563399637347
  (0, 12360)	0.040439126799274694
  (0, 13348)	-0.12131738039782408
  (0, 13655)	-0.020219563399637347
  (0, 14428)	0.08087825359854939
  (0, 15072)	0.08087825359854939
  (0, 15164)	-0.020219563399637347
  (0, 15271)	-0.020219563399637347
  (0, 16260)	0.020219563399637347
  :	:
  (0, 240561)	0.020219563399637347
  (0, 241254)	0.020219563399637347
  (0, 242668)	0.020219563399637347
  (0, 244019)	-0.02021956339963

In [0]:
# CountVectorizer Char
vectorizer = CountVectorizer(min_df=1,
                             max_df = 0.9,
                             analyzer='char_wb',
                             ngram_range = (2,6),
                             max_features = 50000,
                             strip_accents = "ascii",
                             lowercase = True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape)

print(train_vectors[1])

(1008, 50000)
  (0, 12910)	1
  (0, 12913)	1
  (0, 25498)	2
  (0, 5199)	1
  (0, 25493)	1
  (0, 12909)	2
  (0, 25496)	2
  (0, 5196)	1
  (0, 12892)	1
  (0, 12905)	2
  (0, 25492)	3
  (0, 5195)	1
  (0, 12891)	3
  (0, 25490)	3
  (0, 5188)	1
  (0, 12884)	3
  (0, 2505)	1
  (0, 12730)	1
  (0, 2504)	1
  (0, 24109)	1
  (0, 12729)	1
  (0, 2498)	1
  (0, 24108)	1
  (0, 12703)	1
  (0, 43711)	1
  :	:
  (0, 25762)	2
  (0, 13052)	1
  (0, 2338)	2
  (0, 18723)	1
  (0, 6143)	1
  (0, 5605)	1
  (0, 26621)	1
  (0, 13712)	1
  (0, 5187)	1
  (0, 6440)	1
  (0, 36454)	1
  (0, 6439)	2
  (0, 45776)	1
  (0, 36453)	2
  (0, 6438)	2
  (0, 45775)	2
  (0, 36452)	2
  (0, 6436)	2
  (0, 22308)	1
  (0, 2850)	1
  (0, 5603)	2
  (0, 22919)	2
  (0, 26827)	1
  (0, 20460)	2
  (0, 26826)	2


In [79]:
# CountVectorizer Word
vectorizer = CountVectorizer(min_df=1,
                             max_df = 0.7,
                             analyzer='word',
                             ngram_range = (1,1),
                             max_features = 50000,
                             strip_accents = "ascii",
                             lowercase = True)

train_vectors = vectorizer.fit_transform(trainCorpus)
dev_vectors = vectorizer.transform(devCorpus)


print(train_vectors.shape[1])

print(train_vectors[1])

(1008, 4791)
  (0, 2307)	1
  (0, 197)	1
  (0, 880)	1
  (0, 1405)	1
  (0, 3751)	1
  (0, 3691)	1
  (0, 1986)	1
  (0, 2068)	1
  (0, 4265)	1
  (0, 4094)	1
  (0, 1548)	1
  (0, 2693)	1
  (0, 3682)	1
  (0, 3010)	1
  (0, 3688)	1
  (0, 2796)	1


In [41]:
def dict_tweet (tweets):
  res = []
  for tweet in tweets:
    neg = 0;
    pos = 0;

    #print (tweet)
    words = tweet.split(' ')
    for word in words:
      word = word.replace('-', '')
      word = word.replace('\n', '')
      word = word.replace('.', '')
      word = word.replace(',', '')
      word = word.replace('!', '')
      word = word.replace('""', '')
      word = word.replace('#', '')
      word = word.replace('*', '')
      word = word.replace('@', '')
      word = word.lower()
      if word in Poldict:
        if Poldict[word] == "negative":
          neg += 1
        if Poldict[word] == "positive":
           pos += 1
    total=neg+pos
    res.append(pos-neg)
    '''
    if(total>0):
      res.append({'p' : pos/total, 'n' : neg/total})
    else:
      res.append({'p' :0, 'n' : 0})
    '''
  return res
#res = dict_tweet(['-Me caes muy bien \n-Tienes que jugar más partidas al lol con Russel y conmigo\n-Por qué tan Otako, deja de ser otako\n-Haber si me muero', '@myendlesshazza a. que puto mal escribo\n\nb. me sigo surrando help \n\n3. ha quedado raro el "cómetelo" ahí JAJAJAJA', '@estherct209 jajajaja la tuya y la d mucha gente seguro!! Pero yo no puedo sin mi melena me muero '])
res = dict_tweet(trainCorpus)
#print(res)

correcto = 0
incorrecto = 0
neutral = 0
for i in range(0, len(trainCorpus)):
  #print(res[i])
  #print(train_labels[i])
  
  if res[i] < 0 and train_labels[i]=="N":
     correcto += 1
  elif res[i] > 0 and train_labels[i]=="P":
    correcto += 1
  elif res[i] == 0:
    neutral +=1
  else:
    incorrecto += 1
    
print ("Correcto "+str(correcto))
print ("Incorrecto "+str(incorrecto))
print ("Neutral "+str(neutral))


Correcto 401
Incorrecto 261
Neutral 346


In [63]:
text_clf = Pipeline([
    ('vect', TfidfVectorizer(min_df=1,
                             max_df = 0.99,
                             sublinear_tf=False,
                             analyzer='word',
                             ngram_range = (1,1),
                             max_features = None,
                             strip_accents = "ascii",
                             lowercase = True,
                             smooth_idf = True,
                             use_idf=False)),
    
    ('tfidf', TfidfTransformer()),
    ('clf', svm.LinearSVC(penalty='l2',
                          loss='squared_hinge',
                          dual=True,
                          tol=0.0001, 
                          C=1.0, 
                          multi_class='ovr',
                          fit_intercept=True,
                          intercept_scaling=1,
                          class_weight=None,
                          verbose=0,
                          random_state=None,
                          max_iter=1000)),
])

text_clf.fit(trainCorpus, train_labels)  

predicted = text_clf.predict(devCorpus)
np.mean(predicted == dev_labels)            


print(classification_report(dev_labels, predicted))

              precision    recall  f1-score   support

           N       0.59      0.82      0.69       219
         NEU       0.33      0.12      0.17        69
        NONE       0.42      0.18      0.25        62
           P       0.60      0.59      0.59       156

   micro avg       0.57      0.57      0.57       506
   macro avg       0.49      0.43      0.43       506
weighted avg       0.54      0.57      0.53       506



In [42]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class TextStats(BaseEstimator, TransformerMixin):
  
    def fit(self, x, y=None):
        return self

    def transform(self, tweet):
        return dict_tweet(tweet)
                


pipeline =  Pipeline([

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('ngram', Pipeline([
                ('vect', TfidfVectorizer(min_df=1,
                             max_df = 0.99,
                             sublinear_tf=False,
                             analyzer='word',
                             ngram_range = (1,1),
                             max_features = None,
                             strip_accents = "ascii",
                             lowercase = True,
                             smooth_idf = True,
                             use_idf=False)),
                ('tfidf', TfidfTransformer(norm='l2', 
                                           use_idf=True,
                                           smooth_idf=True,
                                           sublinear_tf=False)),
            ])),


            # Pipeline for pulling ad hoc features from post's body
            ('diccionari', Pipeline([
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
            ])),

        ],

        # weight components in FeatureUnion
        transformer_weights={
            'subjngramect': 1.0,
            'diccionari': 0.0,
        },
    )),

    # Use a SVC classifier on the combined features
    ('clf', svm.LinearSVC(penalty='l2',
                          loss='squared_hinge',
                          dual=True,
                          tol=0.0001, 
                          C=1.0, 
                          multi_class='ovr',
                          fit_intercept=True,
                          intercept_scaling=1,
                          class_weight=None,
                          verbose=0,
                          random_state=None,
                          max_iter=1000)),
])


#print (text_clf.named_steps['union'].transformer_list[0][1].named_steps['vect'].get_feature_names())


pipeline.fit(trainCorpus, train_labels)

predicted = pipeline.predict(devCorpus)


print(classification_report(dev_labels, predicted))


AttributeError: ignored

In [0]:
# Perform classification with SVM, kernel=rbf
    classifier_rbf = svm.SVC()
    t0 = time.time()
    classifier_rbf.fit(train_vectors, train_labels)
    t1 = time.time()
    prediction_rbf = classifier_rbf.predict(dev_vectors)
    t2 = time.time()
    time_rbf_train = t1-t0
    time_rbf_predict = t2-t1

    # Perform classification with SVM, kernel=linear
    classifier_linear = svm.SVC(kernel='linear')
    t0 = time.time()
    classifier_linear.fit(train_vectors, train_labels)
    t1 = time.time()
    prediction_linear = classifier_linear.predict(dev_vectors)
    t2 = time.time()
    time_linear_train = t1-t0
    time_linear_predict = t2-t1

    # Perform classification with SVM, kernel=linear
    classifier_liblinear = svm.LinearSVC()
    t0 = time.time()
    classifier_liblinear.fit(train_vectors, train_labels)
    t1 = time.time()
    prediction_liblinear = classifier_liblinear.predict(dev_vectors)
    t2 = time.time()
    time_liblinear_train = t1-t0
    time_liblinear_predict = t2-t1

    # Perform classification with GaussianNB
    dense_train = train_vectors.toarray()
    dense_dev = dev_vectors.toarray()
    classifier_nv = GaussianNB()
    t0 = time.time()
    classifier_nv.fit(dense_train, train_labels)
    t1 = time.time()
    prediction_classifier_nv = classifier_nv.predict(dense_dev)
    t2 = time.time()
    time_gnv_train = t1-t0
    time_gnv_predict = t2-t1
    
    # Perform classification with MultinomialNB
    dense_train = train_vectors.toarray()
    dense_dev = dev_vectors.toarray()
    classifier_mnv = MultinomialNB()
    t0 = time.time()
    classifier_mnv.fit(dense_train, train_labels)
    t1 = time.time()
    prediction_classifier_mnv = classifier_mnv.predict(dense_dev)
    t2 = time.time()
    time_mnv_train = t1-t0
    time_mnv_predict = t2-t1
    
    
    # Perform classification with BernoulliNB
    dense_train = train_vectors.toarray()
    dense_dev = dev_vectors.toarray()
    classifier_bnv = BernoulliNB()
    t0 = time.time()
    classifier_bnv.fit(dense_train, train_labels)
    t1 = time.time()
    prediction_classifier_bnv = classifier_bnv.predict(dense_dev)
    t2 = time.time()
    time_bnv_train = t1-t0
    time_bnv_predict = t2-t1
    
  
    # Print results in a nice table
    print("Results for SVC(kernel=rbf)")
    print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
    print(classification_report(dev_labels, prediction_rbf))
    
    print("Results for SVC(kernel=linear)")
    print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
    print(classification_report(dev_labels, prediction_linear))
    
    print("Results for LinearSVC()")
    print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
    print(classification_report(dev_labels, prediction_liblinear))
    
    print("Results for GaussianNB()")
    print("Training time: %fs; Prediction time: %fs" % (time_gnv_train, time_gnv_predict))
    print(classification_report(dev_labels, prediction_classifier_nv))
    
    print("Results for MultinomialNB()")
    print("Training time: %fs; Prediction time: %fs" % (time_mnv_train, time_mnv_predict))
    print(classification_report(dev_labels, prediction_classifier_mnv))
    
    print("Results for BernoulliNB()")
    print("Training time: %fs; Prediction time: %fs" % (time_bnv_train, time_bnv_predict))
    print(classification_report(dev_labels, prediction_classifier_bnv))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Results for SVC(kernel=rbf)
Training time: 0.191416s; Prediction time: 0.069244s
              precision    recall  f1-score   support

           N       0.43      1.00      0.60       219
         NEU       0.00      0.00      0.00        69
        NONE       0.00      0.00      0.00        62
           P       0.00      0.00      0.00       156

   micro avg       0.43      0.43      0.43       506
   macro avg       0.11      0.25      0.15       506
weighted avg       0.19      0.43      0.26       506

Results for SVC(kernel=linear)
Training time: 0.260540s; Prediction time: 0.064028s
              precision    recall  f1-score   support

           N       0.55      0.87      0.67       219
         NEU       0.50      0.04      0.08        69
        NONE       0.00      0.00      0.00        62
           P       0.57      0.54      0.56       156

   micro avg       0.55      0.55      0.55       506
   macro avg       0.40      0.36      0.33       506
weighted avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
